In [4]:
!pip install weaviate-client==4.6.3

  Using cached weaviate_client-4.6.3-py3-none-any.whl.metadata (3.3 kB)
  Using cached httpx-0.27.0-py3-none-any.whl.metadata (7.2 kB)
  Using cached validators-0.28.1-py3-none-any.whl.metadata (3.6 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 324.9/324.9 kB 5.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 225.1/225.1 kB 13.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.9/5.9 MB 50.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 53.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 4.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 316.6/316.6 kB 18.3 MB/s eta 0:00:00
  Attempting uninstall: protobuf
    Found existing installation: protobuf 3.20.3
    Uninstalling protobuf-3.20.3:
      Successfully uninstalled protobuf-3.20.3
  Attempting uninstall: grpcio
    Found existing installation: grpcio 1.64.1
    Uninstalling grpcio-1.64.1:
      Successfully uninstalled grpcio-1.64.1
  Att

In [5]:
import os
import weaviate
import weaviate.classes as wvc
import weaviate.classes.config as wc

# Set your API keys as environment variables
OPENAI_API_KEY = ""
WEAVIATE_API_KEY = ""
WEAVIATE_CLUSTER_URL = ""

In [9]:
client = weaviate.Client(
    url=WEAVIATE_CLUSTER_URL,
    auth_client_secret=weaviate.AuthApiKey(api_key=WEAVIATE_API_KEY),
    additional_headers={"X-OpenAI-Api-Key": OPENAI_API_KEY})

In [11]:
client.schema.delete_class("Movies")

class_obj = {
    "class": "Movies",
    "vectorizer": "text2vec-openai",
    "moduleConfig": {
        "text2vec-openai": {
            "model": "ada",
            "modelVersion": "002",
            "type": "text"
        }
    }
}

In [12]:
client.schema.create_class(class_obj)

In [15]:
import csv

# Open the CSV file for reading
f = open("/content/movies.csv", "r")
current_movie = None
row_count = 0  # Counter to track the number of rows processed

try:
    with client.batch as batch:
      batch.batch_size=100
      # Create a CSV reader object
      reader = csv.reader(f)
      next(reader)  # Skip header row if present

      # Iterate through each row of data
      for movie in reader:
          if row_count >= 1000:  # Stop after processing 1000 rows
              break

          current_movie = movie
          # Define properties based on the columns in your dataset
          properties = {
              "tmdb_id": movie[1],
              "imdb_id": movie[2],
              "title": movie[3],
              "original_title": movie[4],
              "adult": movie[5],
              "budget": movie[6],
              "genres": movie[7],
              "homepage": movie[8],
              "original_language": movie[9],
              "overview": movie[10],
              "popularity": movie[11],
              "poster_path": movie[12],
              "production_companies": movie[13],
              "production_countries": movie[14],
              "release_date": movie[15],
              "revenue": movie[16],
              "runtime": movie[17],
              "spoken_languages": movie[18],
              "status": movie[19],
              "vote_average": movie[20],
              "vote_count": movie[21],
              "collection_id": movie[22],
              "collection_name": movie[23],
              "collection_poster_path": movie[24],
              "collection_backdrop_path": movie[25],
          }

          # Assuming movie_collection is an initialized database or storage object
          # that has a `data.insert` method for storing each movie entry
          batch.add_data_object(data_object=properties, class_name="Movies")
          row_count += 1  # Increment the counter
except Exception as e:
    print(f"Exception: {e}.")

f.close()
# client.close()


In [23]:
from weaviate.classes.init import AdditionalConfig, Timeout
import json

# Semantic Search
nearText = {
    "concepts": ["murder", "crime", "mystery", "detective", "spy"]
}

response = (
    client.query.get("Movies", [
        "tmdb_id",
        "imdb_id",
        "title",
        "original_title",
        "adult",
        "budget",
        "genres",
        "homepage",
        "original_language",
        "overview",
        "popularity",
        "poster_path",
        "production_companies",
        "production_countries",
        "release_date",
        "revenue",
        "runtime",
        "spoken_languages",
        "status",
        "vote_average",
        "vote_count",
        "collection_id",
        "collection_name",
        "collection_poster_path",
        "collection_backdrop_path"
    ])
    .with_near_text(nearText)
    .with_limit(10)
    .do()
)

print(json.dumps(response, indent=4))

{
    "data": {
        "Get": {
            "Movies": [
                {
                    "adult": "False",
                    "budget": "0",
                    "collection_backdrop_path": "",
                    "collection_id": "",
                    "collection_name": "",
                    "collection_poster_path": "",
                    "genres": "Crime|Drama",
                    "homepage": "null",
                    "imdb_id": "tt0119250",
                    "original_language": "ja",
                    "original_title": "\u306f\u306a\u3073",
                    "overview": "A police officer leaves the force in the face of harrowing personal and professional difficulties. Spiraling into a depression, he makes questionable decisions.",
                    "popularity": "5.044722",
                    "poster_path": "/n0aDY2SIASdgh4g1FJYSH3IUYzz.jpg",
                    "production_companies": "Bandai Visual Company|Office Kitano|TV Tokyo|Tokyo FM Broadcasting Co.",